In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df = pd.read_excel('CEL SCMission 2024 _ Round 3 _ Data (shareable).xlsx', sheet_name=None)
df_sales = df.get('Sales Order')
df_product = df.get('Product Master')

In [18]:
df_merge = pd.merge(df_sales, df_product, left_on='Product ID', right_on='Product ID', how='left')
df_merge['KG per Pallet']=df_merge['KG per Each']*df_merge['Each Per Carton']*df_merge['Carton Per Pallet']
df_merge['No.Pallets'] = df_merge['Quantity in Kg']/df_merge['KG per Pallet']
df_merge['Date'] = pd.to_datetime(df_merge['Date'])
df_merge['Product ID'] = df_merge['Product ID'].astype(str).str.zfill(7)
df_merge = df_merge[df_merge['Quantity in Kg']>0]
df_merge['Trade'] = np.where(df_merge['Country of Origin'] == df_merge['Sales in Country'], 'Domestic', 'International')
df_merge['Month_Year'] = df_merge['Date'].dt.strftime('%B %Y')

# Export data for distribution network

In [60]:
df_merge.to_excel('Combined data master.xlsx', index=False)

# Top product list

In [8]:
writer = pd.ExcelWriter("Top product list.xlsx")
df_product = df_merge.groupby(['Product ID', 'Product Name', 'Country of Origin','Category'])['Quantity in Kg'].sum().reset_index()
for i in df_product['Category'].unique():
    df_top_product = df_product[df_product['Category'] == i].sort_values('Quantity in Kg', ascending=False).head(6)
    if df_top_product.empty:
        continue
    else:
        df_top_product.to_excel(writer, sheet_name=i, index = False, freeze_panes=(1, 0)) 
    pd.io.formats.excel.header_style = None
writer.close()

# ABC Classification

In [14]:
abc_classification = pd.ExcelWriter("ABC Classifiction for each year.xlsx")

df_merge['Year'] = df_merge['Date'].dt.year
list_year = df_merge['Year'].unique()

abc_classifications = {}
for year in list_year:
    product_sales = df_merge[df_merge['Year'] == year]
    product_sales = product_sales.groupby(['Product ID','Product Name'])['Quantity in Kg'].sum().reset_index()
    product_sales_sorted = product_sales.sort_values(by='Quantity in Kg', ascending=False)
    total_sales_volume = product_sales_sorted['Quantity in Kg'].sum()
    product_sales_sorted['Cumulative Volume'] = product_sales_sorted['Quantity in Kg'].cumsum()
    product_sales_sorted['Cumulative Percentage'] = 100 * product_sales_sorted['Cumulative Volume'] / total_sales_volume

    product_sales_sorted['Category'] = pd.cut(product_sales_sorted['Cumulative Percentage'],
                                            bins=[0, 80, 95, 100],
                                            labels=['A', 'B', 'C'])
    product_sales_sorted.to_excel(abc_classification, sheet_name=f'{year}', index = False, freeze_panes=(1, 0)) 
    pd.io.formats.excel.header_style = None
abc_classification.close()

